## Testing stuff out

In [ ]:
!pip install gensim

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim
import os
import warnings
import time



from util import get_data
from util import mapping
# from view_finder.config import data_path

In [ ]:
raw_data_dir = '/home/emily/Documents/ViewFinder/data/raw/'

In [ ]:
# Get data and save to disk

DATA = get_data.search_ridewithgps('Shokan, NY', 25)

In [ ]:
trips, routes = get_data.parse_ridewithgps_search_results(DATA)
trips.to_csv(raw_data_dir + 'ridewgps_trips.csv')
routes.to_csv(raw_data_dir + 'ridewgps_routes.csv')

In [ ]:
for i, r in routes.iterrows():
    get_data.ridewithgps_api_route(r.id)
    time.sleep(1.5)

In [ ]:
# # routes.loc[i]
# for i, r in routes.iterrows():
#     if i == 3979: break
#     if r['pavement_type_id'] + 0 == r['pavement_type_id']:
#         print('{}: pavement {}'.format(r.id, r['pavement_type_id']))
#     if r.highlighted_photo_id:
#         print('{}: photo {}'.format(r.id, r.highlighted_photo_id))

# routes[routes.pavement_type_id > 0].shape
routes.best_for_id.value_counts()
# routes[routes.best_for_id > 0][['id', 'best_for_id']]
# set(routes.columns) - set(trips.columns)

In [ ]:
trips[(trips.highlighted_photo_id > 0) & (trips.distance < 1609 * 50)][['id']]

In [ ]:
rte_id = 41025647#48827366
URL = "http://ridewithgps.com/trips/{}.json".format(rte_id)
r = requests.get(url = URL)
r.json()

In [ ]:
d = r.json()
df = pd.DataFrame(d['track_points'])
(df.iloc[-1].t - df.t[0]) / 60 / 60

# a = pd.read_csv('data/raw/ridewgps/routes/29785085.csv')
# a

In [ ]:
# plt.plot(df.x, df.y, 'k.')
df['t_diff'] = df.t - (list(df.iloc[1:].t.values) + [df.iloc[-1].t])
# [df.iloc[-1].t]
plt.hist(df.t_diff[df.t_diff > -3])

In [ ]:
routes.description.nunique()

In [ ]:
n_scenic = 0
for d in trips.description.unique():
    if d and 'scenic' in d.lower():
        n_scenic += 1
        print(d)
print(n_scenic)
    

In [ ]:
all_description = list(np.concatenate([trips.description.unique(), routes.description.unique(), 
                                  trips.name.unique(), routes.name.unique()]))
all_description = [i for i in all_description if i]

In [ ]:

warnings.filterwarnings("ignore")

class SentenceCorpus(object):

    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for row in self.corpus:
            tokens = row.lower().strip().split()
            yield tokens



sentences = SentenceCorpus(all_description)
model = gensim.models.Word2Vec(sentences, min_count=10, window=10, size=10)

In [ ]:
model.wv.most_similar(positive=["mountain"], topn=10)


In [ ]:
trips['is_trip'] = True
routes['is_trip'] = False
common_cols = [col for col in trips.columns if col in routes.columns]
rides = trips[common_cols].append(routes[common_cols])
rides.distance = mapping.metres_to_miles(rides.distance)
rides = rides[(rides.distance <= 125)]
rides.shape

In [ ]:
common_cols
useful_cols = ['id', 'is_trip', 'distance', 'crow_dist',
              'first_lat', 'first_lng', 'last_lat', 'last_lng', 'circular']

In [ ]:
rides['crow_dist'] = rides.apply(
    lambda x: mapping.dist_lat_lon(x.first_lat, x.first_lng, x.last_lat, x.last_lng), axis=1
)

In [ ]:
rides['circular'] = rides.apply(lambda x: x.crow_dist <= 0.5, axis=1)
rides.circular.value_counts()

In [ ]:
rides[useful_cols].tail(10)

In [ ]:
plt.hist(rides[rides.crow_dist < 1].crow_dist)

In [ ]:
plt.hist(rides.distance - rides.crow_dist)

In [ ]:
rides[(rides.crow_dist > rides.distance)][['id', 'is_trip', 'distance', 'crow_dist', 'first_lat', 'first_lng', 'last_lat', 'last_lng']]#.is_trip.value_counts()
rides[rides.distance > 600][['id', 'is_trip', 'distance', 'crow_dist', 'first_lat', 'first_lng', 'last_lat', 'last_lng']]#.is_trip.value_counts()

In [ ]:
rides_filt = rides[(rides.distance < 125) & (rides.distance >= rides.crow_dist)]
plt.hist(rides_filt.distance - rides_filt.crow_dist)